##  <i class="fa fa-spinner"></i> 装载自定义模块
把当前repo根目录加入 Python 搜索模块的路径列表，并求得根目录

In [ ]:
import os,sys

def find_repo_root():
    # 获取当前脚本的绝对路径
    try:
        # 如果当前运行环境是 Jupyter Notebook，使用当前工作目录
        script_path = os.path.abspath('.')
    except NameError:
        # 否则，使用 __file__ 变量
        script_path = os.path.abspath(__file__)
    # 判断当前脚本目录是否存在`.git`文件夹
    if os.path.isdir(os.path.join(script_path, '.git')):
        return script_path
    # 初始化存储结果的变量
    result = None
    # 循环求当前路径的父目录，直到找到`.git`文件夹
    while True:
        # 将当前路径的父目录赋值给当前路径
        script_path = os.path.dirname(script_path)
        # 判断当前路径是否存在`.git`文件夹
        if os.path.isdir(os.path.join(script_path, '.git')):
            # 如果存在，将当前路径存储在结果变量中
            result = script_path
        # 判断当前路径是否为根目录（即是否已经搜索到最外层）
        if script_path == '/':
            # 如果是，返回结果变量的值
            return result
        
repo_root_dir=find_repo_root()
sys.path.append(repo_root_dir)
print(repo_root_dir)
# 获取repo所在的根目录
root_dir = os.path.dirname(repo_root_dir)
# 获取当前环境名称与当前环境content路径
from func.env import detect_environment
env_cb = detect_environment()
env_name = env_cb['env_name']
content_path = env_cb['content_path']

## <i class="fa fa-arrow-down"></i> 安装必要依赖

In [ ]:
import subprocess

libraries_to_install = [
    
]

tools_to_install = [
    'nmap',
    'net-tools',
    'dnsutils',
    'lsb-release',
    'lshw'
]

installed_libraries = subprocess.run(["pip", "freeze"], capture_output=True).stdout.decode().split("\n")

# 尝试安装所有未安装的库
for library in libraries_to_install:
    if library == "black[jupyter]" and any("black" in installed_library for installed_library in installed_libraries):
        print(f"【 {library} 】已经安装，跳过安装")
        continue
    elif not any(library in installed_library for installed_library in installed_libraries):
        !pip install $library
    else:
        print(f"【 {library} 】已经安装，跳过安装")
        
# 尝试安装所有未安装的工具
update_needed = False
for tool in tools_to_install:
    exit_code = subprocess.run(["dpkg", "-s", tool], capture_output=True).returncode
    if exit_code != 0:
        update_needed = True
        break
    else:
        print(f"【 {tool} 】已经安装，跳过安装")
if update_needed:
    !apt-get update
    for tool in tools_to_install:
        exit_code = subprocess.run(["dpkg", "-s", tool], capture_output=True).returncode
        if exit_code != 0:
            !apt-get install -y $tool

In [ ]:
import subprocess
import re

def get_ip():
    output = subprocess.check_output("ifconfig | grep 'inet'", shell=True).decode()
    ips = output.strip().split('\n')
    ip_list = []
    for ip in ips:
        ip_address = ip.split()[1]
        if not (ip_address == '127.0.0.1' or ip_address == 'localhost'):
            ip_list.append(ip_address)
    return ip_list

def get_hostname(current_ip):
    if not current_ip:
        current_ip = get_ip()
    output = subprocess.check_output(f"nmap {current_ip}", shell=True).decode()
    match = re.search(r"Nmap scan report for (.*)\(", output)
    if match:
        hostname = match.group(1)
    else:
        hostname = "unknown"
    return hostname

def get_nslookup_result(domain):
    output = subprocess.check_output(f"nslookup {domain}", shell=True).decode()
    match = re.search(r"Name:\s+(.*)\n.*Address:\s+(.*)", output)
    if match:
        return match.group(1), match.group(2)
    else:
        return None, None

def get_public_ip():
    return subprocess.check_output("curl --silent ifconfig.me", shell=True).decode().strip()
    
# 调用get_ip函数并返回列表中的第一个有效的IP地址
current_ip = get_ip()[0]
# 调用get_hostname函数,获取本机的hostname
current_hostname = get_hostname(current_ip)
# 调用get_nslookup_result函数获取代理地址域名的网络信息
proxy_hostname, proxy_address = get_nslookup_result("alchemist-experience")
public_ip = get_public_ip()

In [ ]:
import subprocess

#获取操作系统版本
def get_os_version():
    output = subprocess.check_output("lsb_release -a", shell=True).decode()
    os_version = output.strip()
    return os_version

#获取操作系统内核版本
def get_kernel_version():
    output = subprocess.check_output("uname -r", shell=True).decode()
    kernel_version = output.strip()
    return kernel_version

#获取软件包列表
def get_installed_packages():
    output = subprocess.check_output("dpkg -l", shell=True).decode()
    packages = output.strip()
    return packages

#获取硬件配置
def get_hardware_config():
    output = subprocess.check_output("lshw", shell=True).decode()
    hardware_config = output.strip()
    return hardware_config

#获取进程和服务
def get_processes():
    output = subprocess.check_output("ps -ef", shell=True).decode()
    processes = output.strip()
    return processes

#获取网络连接状态
def get_network_connections():
    output = subprocess.check_output("netstat -tuln", shell=True).decode()
    network_connections = output.strip()
    return network_connections

os_version = get_os_version()
kernel_version = get_kernel_version()
installed_packages = get_installed_packages()
hardware_config = get_hardware_config()
processes = get_processes()
network_connections = get_network_connections()


In [ ]:
with open('/output/system_info.txt', 'w') as f:
    # 重定向标准输出
    sys.stdout = f

    # 输出信息
    print('本机IP地址:\n',current_ip)
    print('本机主机名:\n',current_hostname)
    print("代理主机名:\n", proxy_hostname)
    print("代理主机地址:\n", proxy_address)
    print("公网IP:\n",public_ip)

    print('当前操作系统版本:\n',os_version)
    print('当前操作系统内核版本:\n',kernel_version)
    print('获取软件包列表:\n',installed_packages)
    print('获取硬件配置\n:',hardware_config)
    print('获取进程和服务\n:',processes)
    print('获取网络连接状态\n:',network_connections)

# print('本机IP地址：',current_ip)
# print('本机主机名：',current_hostname)
# print("代理主机名:", proxy_hostname)
# print("代理主机地址：", proxy_address)
# print("公网IP：",public_ip)

# print('当前操作系统版本:\n',os_version)
# print('当前操作系统内核版本:\n',kernel_version)
# print('获取软件包列表:\n',installed_packages)
# print('获取硬件配置\n:',hardware_config)
# print('获取进程和服务\n:',processes)
# print('获取网络连接状态\n:',network_connections)

In [ ]:
# import csv

# def save_info_to_csv(file_name):
#     info = [        ['current_ip',current_ip],
#         ['current_hostname',current_hostname],
#         ['proxy_hostname',proxy_hostname],
#         ['proxy_address',proxy_address],
#         ['public_ip',public_ip],
#         ['network_connections', network_connections],
#         ['os_version', os_version],
#         ['kernel_version', kernel_version],
#         ['installed_packages', installed_packages],
#         ['hardware_config', hardware_config],
#         ['processes', processes]
#     ]
#     with open(file_name, 'w') as csv_file:
#         csv_writer = csv.writer(csv_file)
#         csv_writer.writerows(info)

# save_info_to_csv('/output/system_info.csv')
